In [1]:
import pandas as pd
import numpy as np
import torch
src_path = '/Users/Jipeng/PycharmProjects/simulated_multisensory_integration/data/'
file_name = 'simulated_data.csv'
df = pd.read_csv(src_path+file_name)
print (df.shape[0])

10718


## Goal of the module
Training the proprioceptive inference model. Use the two joints angle data to predict the position of the hand.

### Input
shoulder_angle, elbow_angle, elbow_x, elbow_y

What precision should the input be divided into? It depends on the performance. I will explore later.

### Output
spatial_coding of hand position

Depends on the precision of the final coding, the area can be divided into N parts. So there are N position labels. From [1,0,0,...0] to [0,0,0,...1]

### Model structure
input-->F-->FC-->FC
### To do
1. Let the model run, without considering the performance.<br>
a. Convert the hand position into spatial coding.<br>
b. Split the dataset.<br>
c. Use PyTorch to design the model.

2. Augment the data, let it cover more positions.
3. Divide the data into appropriate precision.

In [2]:
'''
# Test the function, delete later.
print (df['hand_x'])
print (np.around(df['hand_x']))
print (df.loc[:,['elbow_angle','shoulder_angle']])
print (df.shape[0])
'''

"\n# Test the function, delete later.\nprint (df['hand_x'])\nprint (np.around(df['hand_x']))\nprint (df.loc[:,['elbow_angle','shoulder_angle']])\nprint (df.shape[0])\n"

In [3]:
loc_map = range(-4,5)
def label_function(x):
    loc_map = range(-4,5)
    return loc_map.index(x)
    
feature_names = df.columns.tolist()
print feature_names

label = np.around(df['hand_x'])
df['label'] = np.around(df['hand_x'])
df['label1'] = df['label'].astype(np.int)
df['label'] = df['label1'].apply(label_function)
#print (df)

['Unnamed: 0', 'elbow_angle', 'elbow_x', 'elbow_y', 'frame_idx', 'hand_x', 'hand_y', 'shoulder_angle', 'target_x', 'video_idx']


In [4]:
from torch.utils.data import Dataset, DataLoader, sampler

class ArmDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return self.dataset.shape[0]
    def __getitem__(self, idx):
        shoulder_angle = self.dataset.loc[idx,'shoulder_angle']
        elbow_angle = self.dataset.loc[idx,'elbow_angle']
        elbow_x = self.dataset.loc[idx,'elbow_x']
        elbow_y = self.dataset.loc[idx,'elbow_y']
        #elbow_angle = self.dataset.loc[idx,'elbow_angle']
        label = self.dataset.loc[idx,'label']
        sample = {'shoulder_angle':shoulder_angle,'elbow_angle':elbow_angle,'elbow_x':elbow_x,'elbow_y':elbow_y,'label':label}
        return sample

In [5]:
import torch.nn as nn
#import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(4,15)
        self.fc2 = nn.Linear(15,20)
        self.fc3 = nn.Linear(20,len(loc_map))
    def forward(self,x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        #x = F.softmax(x,dim=1)
        return x
    
net = Net()

In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001, momentum=0.9)

In [7]:
def train_test_loader(train_ratio,dataset):
    n_data = len(dataset)
    split = int(n_data*train_ratio)
    
    indices = list(range(n_data))
    train_sampler = sampler.SubsetRandomSampler(indices[:split])
    test_sampler = sampler.SubsetRandomSampler(indices[split:])
    
    train_loader = DataLoader(dataset, sampler=train_sampler, shuffle=False, batch_size=4)
    test_loader = DataLoader(dataset, sampler=test_sampler, shuffle=False, batch_size=4)
    
    return train_loader, test_loader

In [8]:
#df = df[df['label']>=0]
#df = df.reset_index(drop=True)
arm_dataset = ArmDataset(df)
# Dataloader for the whole dataset
'''
arm_dataloader = DataLoader(arm_dataset,batch_size=4, shuffle=True)
for idx, batch_samples in enumerate(arm_dataloader):
    shoulder_batches, elbow_batches, label_batches = batch_samples['shoulder_angle'],batch_samples['elbow_angle'],batch_samples['label']
    #print(shoulder_batches,label_batches)
'''

train_loader, test_loader = train_test_loader(0.7,arm_dataset)

loss_list = [1000,1000,1000]
checkpoint = 1
earlystop = True
stop = False
for epoch in range(2):
    running_loss = 0.0
    for idx, batch_samples in enumerate(train_loader, 0):
        shoulder_batches, elbow_batches, ex_batches, ey_batches, label_batches = batch_samples['shoulder_angle'],batch_samples['elbow_angle'],batch_samples['elbow_x'],batch_samples['elbow_y'],batch_samples['label']
        # Clears the gradients of all optimized tensors
        optimizer.zero_grad()
        shoulder_batches = shoulder_batches.view(1,-1)
        elbow_batches = elbow_batches.view(1,-1)
        ex_batches = ex_batches.view(1,-1)
        ey_batches = ey_batches.view(1,-1)
        inputs = torch.cat([shoulder_batches,elbow_batches,ex_batches, ey_batches],dim=0).T
        #print (inputs, label_batches)
        outputs = net(inputs)
        loss = criterion(outputs,label_batches)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 99:
            checkpoint+=1
            loss_list.append(running_loss/100)
            print ('[%d, %5d] loss: %.3f'%
                   (epoch+1, idx+1, running_loss/100))
            running_loss = 0.0
            if loss_list[checkpoint] > loss_list[checkpoint-1] > loss_list[checkpoint-2] > loss_list[checkpoint-3] and earlystop:
                stop = True
                break
        if stop:
            break
    
    print ('Finished Training')

[1,   100] loss: 1.877
[1,   200] loss: 1.579
[1,   300] loss: 1.421
[1,   400] loss: 1.366
[1,   500] loss: 1.337
[1,   600] loss: 1.283
[1,   700] loss: 1.230
[1,   800] loss: 1.271
[1,   900] loss: 1.161
[1,  1000] loss: 1.471
[1,  1100] loss: 1.246
[1,  1200] loss: 1.281
[1,  1300] loss: 1.197
[1,  1400] loss: 1.171
[1,  1500] loss: 1.236
[1,  1600] loss: 1.207
[1,  1700] loss: 1.185
[1,  1800] loss: 1.147
Finished Training
[2,   100] loss: 1.026
[2,   200] loss: 1.062
[2,   300] loss: 1.211
[2,   400] loss: 1.495
[2,   500] loss: 1.265
Finished Training


In [9]:
true_correct = 0
approximate_correct = 0
total = 0
scope = 1
loc_map = range(-4,5)
position_correct = list(0 for i in range(len(loc_map)))
position_total = list(0 for i in range(len(loc_map)))
with torch.no_grad():
    for data in test_loader:
         test_shoulder, test_elbow, test_ex, test_ey, test_label = data['shoulder_angle'],data['elbow_angle'],data['elbow_x'],data['elbow_y'],data['label']
         test_shoulder = test_shoulder.view(1,-1)
         test_elbow = test_elbow.view(1,-1)
         test_ex = test_ex.view(1,-1)
         test_ey = test_ey.view(1,-1)
         test_inputs = torch.cat([test_shoulder,test_elbow,test_ex,test_ey],dim=0).T
         test_outputs = net(test_inputs)
         _,predicted = torch.max(test_outputs.data,1)
         #print(predicted, test_label)
         total += test_label.size(0)
         true_correct += (predicted == test_label).sum().item()
         c = [0 for i in range(4)]
         for i in range(test_label.size(0)):
             if test_label[i]-scope <= predicted[i] <= test_label[i]+scope:
                 position_correct[predicted[i]] += 1
             position_total[predicted[i]] += 1
for i in range(len(position_total)):
    approximate_correct += position_correct[i]
    if position_total[i] == 0:
        print('Accuracy of position %d : %d %%'%(loc_map[i], 0))
    else:
        print('Accuracy of position %2s : %d %% and the total number is %d'%(loc_map[i], 100 * position_correct[i]/position_total[i],position_total[i]))
print( 'The true accuracy of the network is %d %%'%(100 * true_correct/total))
print( 'The approximate accuracy of the network is %d %%'%(100 * approximate_correct/total))


Accuracy of position -4 : 0 %
Accuracy of position -3 : 0 %
Accuracy of position -2 : 0 %
Accuracy of position -1 : 0 %
Accuracy of position  0 : 59 % and the total number is 944
Accuracy of position  1 : 55 % and the total number is 2272
Accuracy of position 2 : 0 %
Accuracy of position 3 : 0 %
Accuracy of position 4 : 0 %
The true accuracy of the network is 31 %
The approximate accuracy of the network is 56 %


In [18]:
arm_dataloader = DataLoader(arm_dataset,batch_size=1, shuffle=False)
with torch.no_grad():
    for idx, batch_samples in enumerate(arm_dataloader):
        shoulder_batches, elbow_batches, ex_batches, ey_batches, label_batches = batch_samples['shoulder_angle'],batch_samples['elbow_angle'],batch_samples['elbow_x'],batch_samples['elbow_y'],batch_samples['label']        #print(shoulder_batches,label_batches)
        optimizer.zero_grad()
        shoulder_angle = shoulder_batches.view(1,-1)
        elbow_angle = elbow_batches.view(1,-1)
        ex = ex_batches.view(1,-1)
        ey = ey_batches.view(1,-1)
        inputs = torch.cat([shoulder_angle,elbow_angle,ex, ey],dim=0).T
        #print (inputs, label_batches)
        outputs = net(inputs)
        l1 = outputs.tolist()
        df.loc[idx,'priopredict'] = ' '.join(str(i) for i in l1[0])

In [21]:
print df['priopredict']
df.to_csv('/Users/Jipeng/PycharmProjects/simulated_multisensory_integration/data/after_prio.csv')

0        -0.60513406992 0.565839469433 1.33064591885 0....
1        -0.605134010315 0.565839529037 1.33064591885 0...
2        -0.60513406992 0.565839469433 1.33064591885 0....
3        -0.605134129524 0.565839350224 1.33064568043 0...
4        -0.60513406992 0.565839469433 1.33064591885 0....
5        -0.605134487152 0.565838515759 1.33064472675 0...
6        -0.60513406992 0.565839469433 1.33064591885 0....
7        -0.605134010315 0.565839529037 1.33064591885 0...
8        -0.605134010315 0.565839529037 1.33064591885 0...
9        -0.605134010315 0.565839529037 1.33064591885 0...
10       -0.605134010315 0.565839529037 1.33064591885 0...
11       -0.605134010315 0.565839529037 1.33064591885 0...
12       -0.605134010315 0.565839529037 1.33064591885 0...
13       -0.605134010315 0.565839529037 1.33064591885 0...
14       -0.605134010315 0.565839529037 1.33064591885 0...
15       -0.605134010315 0.565839529037 1.33064591885 0...
16       -0.605134010315 0.565839529037 1.33064591885 0.